In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error

In [2]:
df_serie_hist = pd.read_csv(r'C:\Users\DuduCuco\Documents\NewDoencasRespiratorias\DoencasRespiratorias\Scripts\serie_hist.csv')

In [4]:
# Função para preparar a visão de internações por mês/ano de um hospital específico
def prepare_time_series_for_hospital(data, hospital_name):
    # Filtrar o dataframe pelo hospital desejado
    data_hospital = data[data['Hospital'].str.contains(hospital_name, na=False, case=False)]
    
    # Se não encontrar o hospital, retorne None
    if data_hospital.empty:
        print(f"Hospital '{hospital_name}' não encontrado.")
        return None
    
    # Transformando o índice
    data_hospital.set_index(['Ano', 'Mês'], inplace=True)
    data_hospital.sort_index(inplace=True)
    
    # Selecionar a coluna de internações
    data_hospital = data_hospital['Internações']
    
    # Definir uma frequência mensal para o índice
    data_hospital.index = pd.date_range(start=f"{data_hospital.index[0][0]}-{data_hospital.index[0][1]}", periods=len(data_hospital), freq='M')
    
    return data_hospital

# Função de previsão usando SARIMAX com auto_arima
def forecast_time_series(data, steps=12):
    # Use auto_arima para encontrar os melhores hiperparâmetros do modelo SARIMA
    model = auto_arima(data, seasonal=True, m=12, stepwise=True, trace=True, error_action='ignore',
                      suppress_warnings=True, max_order=None)
    
    # Ajustar o modelo SARIMAX com os hiperparâmetros encontrados
    sarimax_model = SARIMAX(data, order=model.order, seasonal_order=model.seasonal_order)
    fitted_model = sarimax_model.fit(disp=False)
    
    # Fazer previsões
    forecast = fitted_model.get_forecast(steps=steps)
    forecast_index = pd.date_range(data.index[-1] + DateOffset(months=1), periods=steps, freq='M')
    forecast_values = forecast.predicted_mean
    forecast_conf_int = forecast.conf_int()
    
    # Construir dataframe de previsão
    forecast_df = pd.DataFrame({
        'Forecast': forecast_values,
        'Lower CI': forecast_conf_int.iloc[:, 0],
        'Upper CI': forecast_conf_int.iloc[:, 1]
    }, index=forecast_index)
    
    return forecast_df

# Função de previsão usando Prophet
def forecast_time_series_prophet(data):
    # Preparar o dataframe para o Prophet
    data_prophet = pd.DataFrame({
        'ds': data.index,
        'y': data.values
    })
    
    # Ajustar o modelo Prophet
    prophet_model = Prophet()
    prophet_model.fit(data_prophet)
    
    # Fazer previsões
    future = prophet_model.make_future_dataframe(periods=12, freq='M')
    forecast = prophet_model.predict(future)
    
    return forecast.set_index('ds')['yhat']

# Função para calcular o MSE
def calculate_mse(actual, predicted):
    return mean_squared_error(actual, predicted)

# Carregar o arquivo CSV
data = pd.read_csv(r'C:\Users\DuduCuco\Documents\NewDoencasRespiratorias\DoencasRespiratorias\Scripts\serie_hist.csv')

# Obter os nomes únicos dos hospitais
hospital_names = data['Hospital'].unique()

# Criar um DataFrame vazio para armazenar as previsões
forecast_df = pd.DataFrame()

# Iterar sobre os nomes dos hospitais e fazer previsões para cada um
for hospital_name in hospital_names:
    print(f"Previsão para o hospital '{hospital_name}':")
    # Preparar a série temporal para o hospital específico
    hospital_time_series = prepare_time_series_for_hospital(data, hospital_name)

    # Se hospital_time_series não for None, faça a previsão
    if hospital_time_series is not None:
        # Previsão com SARIMAX
        hospital_forecast_sarimax = forecast_time_series(hospital_time_series)
        # Previsão com Prophet
        hospital_forecast_prophet = forecast_time_series_prophet(hospital_time_series)
        
        # Calcular o MSE para cada previsão
        mse_sarimax = calculate_mse(hospital_time_series[-12:], hospital_forecast_sarimax['Forecast'])
        mse_prophet = calculate_mse(hospital_time_series[-12:], hospital_forecast_prophet[-12:])
        
        # Imprimir os MSEs
        print(f"MSE do modelo SARIMAX: {mse_sarimax}")
        print(f"MSE do modelo Prophet: {mse_prophet}")
        
        # Comparar os MSEs e escolher o melhor modelo
        if mse_sarimax < mse_prophet:
            print("O modelo SARIMAX é o melhor para este hospital.")
            best_forecast = hospital_forecast_sarimax['Forecast']
        else:
            print("O modelo Prophet é o melhor para este hospital.")
            best_forecast = hospital_forecast_prophet[-12:]
        
        # Adicionar as previsões do melhor modelo ao DataFrame de previsões
        forecast_df[hospital_name] = best_forecast

        # Plotar as previsões
        plt.figure(figsize=(10, 6))
        plt.plot(hospital_time_series, label='Dados originais')
        plt.plot(hospital_forecast_sarimax['Forecast'], label=f'Previsão SARIMAX (MSE: {mse_sarimax:.2f})')
        plt.plot(hospital_forecast_prophet[-12:], label=f'Previsão Prophet (MSE: {mse_prophet:.2f})')
        plt.legend()
        plt.title(f"Previsões para o hospital '{hospital_name}'")
        plt.show()

# Imprimir o DataFrame de previsões
print(forecast_df)

Previsão para o hospital '2077531 A C CAMARGO CANCER CENTER':


C:\Users\DuduCuco\AppData\Local\Temp\ipykernel_18924\2442496684.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hospital.sort_index(inplace=True)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=2087.895, Time=2.85 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=2150.691, Time=0.06 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=2117.224, Time=0.17 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=2088.480, Time=0.28 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=2148.691, Time=0.02 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=2089.307, Time=0.51 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=2089.321, Time=0.48 sec
 ARIMA(2,1,2)(2,0,1)[12] intercept   : AIC=2088.930, Time=1.88 sec
 ARIMA(2,1,2)(1,0,2)[12] intercept   : AIC=2089.156, Time=1.50 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=2087.469, Time=0.42 sec
 ARIMA(1,1,2)(0,0,0)[12] intercept   : AIC=2086.063, Time=0.44 sec
 ARIMA(1,1,2)(1,0,0)[12] intercept   : AIC=2087.921, Time=0.65 sec
 ARIMA(1,1,2)(0,0,1)[12] intercept   : AIC=2087.909, Time=0.56 sec
 ARIMA(1,1,2)(1,0,1)[12] intercept   : AIC=inf, Time=0.65 sec
 ARIMA(0,1,2)(0,0,0)[12]

AttributeError: 'Prophet' object has no attribute 'stan_backend'